# Data Wrangling

Step one is load up all the libraries I'll need for wrangling.

In [1]:
import pandas as pd
import numpy as np
import warnings
import re
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop = stopwords.words('english')
from library.sb_utils import save_file
warnings.filterwarnings("ignore")

Next, I'll load the data. Right now, the data is split into reviews and products, and I need to consolidate them into a single dataframe.

In [2]:
reviews = pd.read_csv('../data/combined/reviews.csv')
products = pd.read_csv('../data/combined/products.csv')

## Creating A Single DataFrame

So there are two different dataframes to work with. They are `reviews` and `products`. Unsurprisingly, these both correspond to the individual reviews and the details of each flavor. I'll start by looking at the first few rows of each dataframe.

In [3]:
reviews.head(3)

,brand,key,author,date,stars,title,helpful_yes,helpful_no,text,taste,ingredients,texture,likes
0,bj,0_bj,Ilovebennjerry,2017-04-15,3,Not enough brownies!,10.0,3.0,"Super good, don't get me wrong. But I came for...",NaN,NaN,NaN,NaN
1,bj,0_bj,Sweettooth909,2020-01-05,5,I’m OBSESSED with this pint!,3.0,0.0,I decided to try it out although I’m not a hug...,NaN,NaN,NaN,NaN
2,bj,0_bj,LaTanga71,2018-04-26,3,My favorite...More Caramel Please,5.0,2.0,My caramel core begins to disappear about half...,NaN,NaN,NaN,NaN


In [29]:
reviews.head(3).to_markdown(index=False)

"| brand   | key   | author         | date       |   stars | title                             |   helpful_yes |   helpful_no | text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |   taste |   ingredients |   texture |   likes |\n|:--------|:------|:---------------|:-----------|--------:|:----------------------------------|----------

In [5]:
products.head(3)

,brand,key,name,subhead,description,rating,rating_count,ingredients
0,bj,0_bj,Salted Caramel Core,Sweet Cream Ice Cream with Blonde Brownies & a...,Find your way to the ultimate ice cream experi...,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,1_bj,Netflix & Chilll'd™,Peanut Butter Ice Cream with Sweet & Salty Pre...,There’s something for everyone to watch on Net...,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,2_bj,Chip Happens,A Cold Mess of Chocolate Ice Cream with Fudge ...,Sometimes “chip” happens and everything’s a me...,4.7,130,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."


So I can already see some overlapping information as well as informationthat is different in each dataframe. I also see that there are some columns that don't interest me for this project. So, to better prepare the data, I need to get all the columns I want from both dataframes and put it into a single dataframe. Before I create any new dataframes, I need to make sure that the the keys align.

In [6]:
set(products['key']) == set(reviews['key'])

True

Perfect! Now I can perform the merge. I'll store all the columns that I want into a dataframe called `data`.

In [7]:
data = pd.DataFrame()
text = []
title = []
brand = []
author = []
names = []
stars = []
ratings = []
rating_counts = []

In [8]:
for i in range(len(reviews)):
    text.append(reviews['text'].iloc[i])
    title.append(reviews['title'].iloc[i])
    brand.append(reviews['brand'].iloc[i])
    author.append(reviews['author'].iloc[i])
    stars.append(reviews['stars'].iloc[i])
    key = reviews['key'].iloc[i]
    products_key = products[products['key'] == key]
    names.append(products_key['name'].values[0])
    ratings.append(products_key['rating'].values[0])
    rating_counts.append(products_key['rating_count'].values[0])

In [9]:
data['author'] = author
data['brand'] = brand
data['name'] = names
data['rating'] = ratings
data['rating_count'] = rating_counts
data['stars'] = stars
data['title'] = title
data['text'] = text

Now we take a peek at the data and it's shape.

In [10]:
data.head(3)

,author,brand,name,rating,rating_count,stars,title,text
0,Ilovebennjerry,bj,Salted Caramel Core,3.7,208,3,Not enough brownies!,"Super good, don't get me wrong. But I came for..."
1,Sweettooth909,bj,Salted Caramel Core,3.7,208,5,I’m OBSESSED with this pint!,I decided to try it out although I’m not a hug...
2,LaTanga71,bj,Salted Caramel Core,3.7,208,3,My favorite...More Caramel Please,My caramel core begins to disappear about half...


In [11]:
data.shape

(21674, 8)

## Inspecting Nulls And Data Types

Now that there is one complete dataset, I can start checking out some of the data. I'll start with looking at some of the data types.

In [12]:
data.dtypes

author           object
brand            object
name             object
rating          float64
rating_count      int64
stars             int64
title            object
text             object
dtype: object

All these data types seem okay to me. Now, I can check the null values.

In [13]:
for column in data:
    print(column + ": ", sum(data[column].isnull()))

author:  800
brand:  0
name:  0
rating:  0
rating_count:  0
stars:  0
title:  5399
text:  0


So there are quite a few nulls in `title` and in `author`. That is okay. I wasn't really planning on using `author` anyways, I just have it for reference. As for `title`, that is alright too. The key data lives in `text`, which, luckily, doesn't contain any nulls. To deal with the nulls, I'll just fill them with empty strings.

In [14]:
data['title'].fillna('', inplace = True)

## A Little Text Preprocessing

I have two main columns with substantial textual data, `title` and `text`. For EDA and modelling, it is best to combine these into a single column. After that is done, I can remove stopwords, make everything lowercase and remove any special characers.

In [15]:
data['text'] = data[['title', 'text']].apply(' '.join, axis=1)

In [16]:
data.drop(['title'], axis=1, inplace=True)

In [17]:
data.head(3)

,author,brand,name,rating,rating_count,stars,text
0,Ilovebennjerry,bj,Salted Caramel Core,3.7,208,3,"Not enough brownies! Super good, don't get me ..."
1,Sweettooth909,bj,Salted Caramel Core,3.7,208,5,I’m OBSESSED with this pint! I decided to try ...
2,LaTanga71,bj,Salted Caramel Core,3.7,208,3,My favorite...More Caramel Please My caramel c...


I will also clean the text up a bit and remove the stop words, so I can have a more usable text column.

In [18]:
def cleanText(text):
    text = re.sub(r'https?:\/\/\S+', '', text) # remove links
    text = re.sub(r'@[A-Za-z0-0]+', '', text) # remove @
    text = re.sub(r'[!@#$%^&*()_\-+=}{[\]|,.?<>:;\'’`~]', '', text) # remove special characters
    text = re.sub(r'[0-9]+[a-zA-Z]*', '', text) # remove numbers
    return text

In [19]:
data['text'] = data['text'].apply(cleanText)

In [20]:
data['text'] = data['text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

I'll also use a tokenizer and get the lemma from each word. This should make the modelling and sentiment analysis a bit easier.

In [21]:
tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

In [22]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)]

In [23]:
data['text'] = data['text'].apply(lemmatize_text)

In [24]:
data['text'] = [' '.join(map(str, l)) for l in data['text']]

In [25]:
data.head(3)

,author,brand,name,rating,rating_count,stars,text
0,Ilovebennjerry,bj,Salted Caramel Core,3.7,208,3,not enough brownie super good dont get wrong b...
1,Sweettooth909,bj,Salted Caramel Core,3.7,208,5,im obsessed pint i decided try although im hug...
2,LaTanga71,bj,Salted Caramel Core,3.7,208,3,my favoritemore caramel please my caramel core...


One last step. I am going to do a bit of feature engineering here. 

Ratings are given stars on a scale from 1 to 5. This would therefore give 5 different potential classes for each record. This makes this problem into a multi classification problem. I am not sure yet if I want to do a multi classification problem or a binary classification problem yet, so I am going to create a new column, `good_review`, that will take the value `Good` if `stars` is 4 or 5, and `Bad` otherwise

In [26]:
data['good_review'] = np.where((data.stars == 4) | (data.stars==5), "Good", "Bad")

In [27]:
data.head(4)

,author,brand,name,rating,rating_count,stars,text,good_review
0,Ilovebennjerry,bj,Salted Caramel Core,3.7,208,3,not enough brownie super good dont get wrong b...,Bad
1,Sweettooth909,bj,Salted Caramel Core,3.7,208,5,im obsessed pint i decided try although im hug...,Good
2,LaTanga71,bj,Salted Caramel Core,3.7,208,3,my favoritemore caramel please my caramel core...,Bad
3,chicago220,bj,Salted Caramel Core,3.7,208,5,obsessed why people complaining blonde brownie...,Good


Great! This looks like a pretty solid data set to kick things off with. I'll save this as a csv and move on to the EDA.

# Save Data

In [28]:
save_file(data, 'ice_cream_data.csv', '../data')

A file already exists with this name.

Do you want to overwrite? (Y/N)N

Please re-run this cell with a new filename.
